<a href="https://colab.research.google.com/github/MargaridaVitolo/Educ360/blob/main/Semana_7_Margarida_Visualilzacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import plotly.express as px
import streamlit as st

def card_com_borda(titulo, valor):
    st.markdown(
        f"""
        <div style="
            border: 2px solid #00008B;
            border-radius: 10px;
            padding: 10px;
            text-align: center;
            background-color: #f9f9f9;">
            <h3 style="margin: 0;">{titulo}</h3>
            <p style="font-size: 24px; font-weight: bold; margin: 5px 0 0 0;">{valor}</p>
        </div>
        """, unsafe_allow_html=True
    )

def formata_brasileiro(numero):
    saida = f"{numero:_.2f}"
    saida = saida.replace('.',',').replace('_','.')
    return saida

page_bg_img = '''
<style>
    .stApp {
        background-color: #F0F0F0;
    }
</style>
'''

st.markdown(page_bg_img, unsafe_allow_html=True)

st.set_page_config(page_title="🎲 Dados Informática Ltda", layout="wide")

#CSV
df=pd.read_csv('vendas.csv',
               parse_dates=['data_venda'])


# --- Barra Lateral (Filtros) ---
st.sidebar.header("🔍 Filtros")

#Combo das datas
# recebe todas as datas (df), classifica e filtra somente 1 de cada (unique)
datas_unicas = sorted(df['data_venda'].dt.strftime("%d-%m-%Y").unique())
#usando os recursos do st, crio uma barra lateral com as opções (selectbox) com a primeira opçao como Todas
opcao_data = st.sidebar.selectbox("Selecione a data:",options=["Todas"] + datas_unicas)

#Combo de categorias

categorias = sorted(df['categoria'].unique())
opcao_categoria = st.sidebar.selectbox("Selecione uma categoria:", options=["Todas"] + categorias)

#Aplicar filtros

df_filtrado = df.copy()

if opcao_data != "Todas":
    df_filtrado = df_filtrado[df_filtrado["data_venda"].dt.strftime("%d-%m-%Y") == opcao_data]

if opcao_categoria != "Todas":
    df_filtrado = df_filtrado[df_filtrado["categoria"] == opcao_categoria]

# Cálculos para os cards
total_vendas = df_filtrado['quantidade'].sum()
total_vendas_$ = df_filtrado['valor_total'].sum() if 'valor_total' in df_filtrado.columns else 0.
total_vendas_$ = formata_brasileiro(total_vendas_$)
fat_cat = df.groupby('categoria')['valor_total'].mean()
fat_cat = formata_brasileiro(fat_cat.iloc[0])


# Exibir os cards lado a lado
card1, card2, card3 = st.columns(3)
with card1:
    card_com_borda("Nr Vendas", total_vendas)
with card2:
    card_com_borda("Faturamento Total Período", f"R$ {total_vendas_$}")
with card3:
    card_com_borda("Faturamento Medio", f"R$ {fat_cat}")

st.markdown("---")

'''
# Composição dos gráficos

st.title(":blue[🩺 Painel de Consultas Médicas]")
multi = '''Explore as informações das consultas do último período. Utilize os filtros à esquerda para refinar sua análise.'''
st.markdown(multi)
col1, col2, col3 = st.columns(3)

# Grafico 1 - Número de Consultas por Unidade

consultas_unidade = df_filtrado.groupby("unidade").size().reset_index(name="Total")

fig1 = px.bar(
    consultas_unidade,
    x = "unidade",
    y = "Total",
    color="unidade",
    title=f"Nr de Consultas por Unidades<br>({opcao_data})",
    text="Total",
    color_discrete_sequence = px.colors.qualitative.Pastel
)

fig1.update_layout(xaxis_title="Unidade(s)", yaxis_title="Total de Consultas")
fig1.update_layout(title_x=0.5, title_xanchor='center')
col1.plotly_chart(fig1, use_container_width=True)

# Grafico 2 - Consulta por Unidade x Especialidades

consulta_tipo = df_filtrado["tipoconsulta"].value_counts().reset_index(name="Total")
consulta_tipo.columns = ['tipoconsulta', 'Total']

fig2 = px.pie(
    consulta_tipo,
    names = "tipoconsulta",
    values = "Total",
    title = f"Consultas por Especialidade",
    color = "tipoconsulta",
    hole=0.3,
    color_discrete_sequence = px.colors.qualitative.Pastel
)

fig2.update_layout(title_x=0.5, title_xanchor='center')
col2.plotly_chart(fig2, use_container_width=True)

# Grafico 3 - Faturamento Total por Unidade
fat_unidade = df_filtrado.groupby("unidade").agg({"valor": 'sum'}).reset_index()
fat_unidade.columns = ['unidade', 'valor']

fig3 = px.bar(
    fat_unidade,
    x = "unidade",
    y = "valor",
    color = "unidade",
    title=f"Fat. Total por unidade<br>({opcao_formatada})",
    text="valor",
    color_discrete_sequence = px.colors.qualitative.Pastel
)
fig3.update_layout(xaxis_title="Unidade(s)", yaxis_title="Faturamento (R$)")
fig3.update_layout(title_x=0.5, title_xanchor='center')
col3.plotly_chart(fig3, use_container_width=True)

st.markdown("---")

# Visão dos atendimentos

st.subheader(f":blue[🗃️ Registros: {opcao_unidade}]")

# Mostrar a data de retorno - ao invés da quantidade de dias
df_filtrado['data_retorno'] = df_filtrado['dataconsulta'] + pd.to_timedelta(df_filtrado['retornodaconsulta'], unit='days')
df_filtrado = df_filtrado.drop('retornodaconsulta', axis=1)

# Formatar o valor para apresentar virgula no lugar do ponto
df_filtrado['valor_formatado'] = df_filtrado['valor'].apply(formata_brasileiro)
df_filtrado = df_filtrado.drop('valor', axis=1)


coluna_formatada = {
    'dataconsulta': st.column_config.DateColumn("Data Consulta", format='DD/MM/YYYY'),
    'unidade': st.column_config.TextColumn("Unidade"),
    'tipoconsulta': st.column_config.TextColumn("Especialidade"),
    'valor_formatado': st.column_config.TextColumn("Valor (R$)"),
    'data_retorno': st.column_config.DateColumn("Data retorno", format='DD/MM/YYYY')

}
st.dataframe(df_filtrado, column_config=coluna_formatada, use_container_width= True'
''


SyntaxError: unterminated triple-quoted string literal (detected at line 170) (ipython-input-3606616712.py, line 89)